In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
os.chdir("D:\\webscraping_practice")

In [22]:
os.listdir()

['Combined_ucl.xlsx',
 'Fake_jobs_scraped.csv',
 'full_top_scorers.xlsx',
 'main_ucl.xlsx',
 'merging_practice.ipynb',
 'missing_topscorers_tfm_scraped.ipynb',
 'missing_top_scorers.xlsx',
 'Selenium',
 'ucl_finals_analysis.ipynb',
 'ucl_finals_scraped.xlsx',
 'ucl_top_scorers.xlsx']

In [3]:
df=pd.read_excel("main_ucl.xlsx")

#### Feature engineering

In [6]:
df.sample(5)

,Season,Home_team,FT_score,Away_team,Winning_team,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
64,59/60,Real Madrid,7:3,Eintracht Frankfurt,Real Madrid,Spain,West Germany,Scotland,Hampden Park,Ferenc Puskás,Real Madrid,Hungary,12,Centre-Forward
31,92/93,Olympique Marseille,1:0,AC Milan,Olympique Marseille,France,Italy,Germany,Olympiastadion,Romário,PSV Eindhoven,Brazil,7,Centre-Forward
46,77/78,Liverpool FC,1:0,Club Brugge KV,Liverpool FC,England,Belgium,England,Wembley Stadium,Allan Simonsen,Borussia Mönchengladbach,Denmark,5,Second Striker
44,79/80,Nottingham Forest,1:0,Hamburger SV,Nottingham Forest,England,West Germany,Spain,Santiago Bernabéu,Sören Lerby,Ajax Amsterdam,Denmark,10,Central Midfield
1,22/23,Manchester City,1:0,Inter Milan,Manchester City,England,Italy,Turkey,Atatürk Olympic Stadium,Erling Haaland,Manchester City,Norway,12,Centre-Forward


In [7]:
df.replace('West Germany', 'Germany', inplace=True)

Replacing all of occurences "West Germany" as "Germany" to make our analysis more accurate.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Season               69 non-null     object
 1   Home_team            69 non-null     object
 2   FT_score             69 non-null     object
 3   Away_team            69 non-null     object
 4   Winning_team         69 non-null     object
 5   Winners_country      69 non-null     object
 6   Runner_ups_country   69 non-null     object
 7   Host_Country         69 non-null     object
 8   Finals_stadium       69 non-null     object
 9   Top_scorer           69 non-null     object
 10  Top_scorer_club      69 non-null     object
 11  Top_scorer_country   69 non-null     object
 12  Top_scorer_goals     69 non-null     int64 
 13  Top_scorer_position  69 non-null     object
dtypes: int64(1), object(13)
memory usage: 7.7+ KB


In [9]:
df["GD"]=df['FT_score'].apply(
    lambda x: abs(
        int(x.split(' ')[0].split(':')[0]) - int(x.split(' ')[0].split(':')[1])
    )
)
#calculating goal difference

In [10]:
df["Losing_team"] = df.apply(
    lambda row: row["Home_team"] if int(row["FT_score"].split(":")[0]) < int(row["FT_score"].split(":")[1][0]) 
    else row["Away_team"], axis=1
)

In [11]:
df=df[["Season",  "Home_team",	"FT_score",	"Away_team","Winning_team", "Losing_team" ,"GD","Winners_country","Runner_ups_country",	"Host_Country",	"Finals_stadium",	"Top_scorer",	"Top_scorer_club",	"Top_scorer_country",	"Top_scorer_goals",	"Top_scorer_position"]]

In [12]:
df.loc[df.isna().any(axis=1)]

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position


#### Club level analysis:
- Most UCL titles
- Most Runner's up
- Most appearences in Finals
- Best Win % in Finals

Finding the top 5 clubs with most ucls

In [18]:
df["Winning_team"].value_counts().head() 

Real Madrid      15
AC Milan          7
Bayern Munich     6
Liverpool FC      6
FC Barcelona      5
Name: Winning_team, dtype: int64

Finding the top 5 clubs with most runner's up

In [19]:
df["Losing_team"].value_counts().head()

Juventus FC      7
SL Benfica       5
Bayern Munich    5
AC Milan         4
Liverpool FC     4
Name: Losing_team, dtype: int64

Team with most appearences in UCL finals

In [20]:
(df["Winning_team"]
.value_counts()
.add(df["Losing_team"].value_counts(), fill_value=0)
.astype(int).sort_values(ascending=False).head(6))
#add()is used to add two series together,
# fill_value=0 replaces NaN values with 0(since some teams dont appear in winning_team/losing_team col) before adding.
# total_count.head(6)


Real Madrid      18
AC Milan         11
Bayern Munich    11
Liverpool FC     10
Juventus FC       9
FC Barcelona      7
dtype: int32

Calculating Win percentage in finals

In [21]:
(df["Winning_team"].value_counts()
.div(df["Winning_team"].value_counts().add(df["Losing_team"].value_counts(), fill_value=0))
.fillna(0)
.sort_values(ascending=False)
.mul(100)
.round(2)
)

FC Porto                    100.00
PSV Eindhoven               100.00
Nottingham Forest           100.00
RV & AV Feijenoord          100.00
Aston Villa                 100.00
Red Star Belgrade           100.00
Real Madrid                  83.33
FC Barcelona                 71.43
Chelsea FC                   66.67
Ajax Amsterdam               66.67
AC Milan                     63.64
Manchester United            60.00
Liverpool FC                 60.00
Bayern Munich                54.55
Inter Milan                  50.00
Olympique Marseille          50.00
Hamburger SV                 50.00
Celtic FC                    50.00
Steaua Bucharest             50.00
Manchester City              50.00
Borussia Dortmund            33.33
SL Benfica                   28.57
Juventus FC                  22.22
Panathinaikos FC              0.00
Paris Saint-Germain           0.00
Stade Reims                   0.00
Tottenham Hotspur             0.00
UC Sampdoria                  0.00
AC Fiorentina       

#### Finding out instances where clubs won/lost the finals hosted in their Home country

In [22]:
df.head()

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
0,23/24,Borussia Dortmund,0:2,Real Madrid,Real Madrid,Borussia Dortmund,2,Spain,Germany,England,Wembley Stadium,"Harry Kane, Kylian Mbappé","Bayern Munich, Paris Saint-Germain","England, France",8,"Centre-Forward, Centre-Forward"
1,22/23,Manchester City,1:0,Inter Milan,Manchester City,Inter Milan,1,England,Italy,Turkey,Atatürk Olympic Stadium,Erling Haaland,Manchester City,Norway,12,Centre-Forward
2,21/22,Liverpool FC,0:1,Real Madrid,Real Madrid,Liverpool FC,1,Spain,England,France,Stade de France,Karim Benzema,Real Madrid,France,15,Centre-Forward
3,20/21,Manchester City,0:1,Chelsea FC,Chelsea FC,Manchester City,1,England,England,Portugal,Estádio do Dragão,Erling Haaland,Borussia Dortmund,Norway,10,Centre-Forward
4,19/20,Paris Saint-Germain,0:1,Bayern Munich,Bayern Munich,Paris Saint-Germain,1,Germany,France,Portugal,Estádio da Luz,Robert Lewandowski,Bayern Munich,Poland,15,Centre-Forward


In [23]:
df[df["Winners_country"]==df["Host_Country"]].reset_index(drop=True)

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
0,96/97,Borussia Dortmund,3:1,Juventus FC,Borussia Dortmund,Juventus FC,2,Germany,Italy,Germany,Olympiastadion,Milinko Pantic,Atlético de Madrid,Serbia,5,Attacking Midfield
1,95/96,Juventus FC,5:3 on pens,Ajax Amsterdam,Juventus FC,Ajax Amsterdam,2,Italy,Netherlands,Italy,Stadio Olimpico,Jari Litmanen,Ajax Amsterdam,Finland,9,Attacking Midfield
2,77/78,Liverpool FC,1:0,Club Brugge KV,Liverpool FC,Club Brugge KV,1,England,Belgium,England,Wembley Stadium,Allan Simonsen,Borussia Mönchengladbach,Denmark,5,Second Striker
3,71/72,Ajax Amsterdam,2:0,Inter Milan,Ajax Amsterdam,Inter Milan,2,Netherlands,Italy,Netherlands,De Kuip,Silvester Takac,Standard Liège,Serbia,5,Centre-Forward
4,67/68,Manchester United,4:1 AET,SL Benfica,Manchester United,SL Benfica,3,England,Portugal,England,Wembley Stadium,Eusébio,SL Benfica,Portugal,6,Centre-Forward
5,64/65,Inter Milan,1:0,SL Benfica,Inter Milan,SL Benfica,1,Italy,Portugal,Italy,San Siro,Eusébio,SL Benfica,Portugal,9,Centre-Forward
6,56/57,Real Madrid,2:0,AC Fiorentina,Real Madrid,AC Fiorentina,2,Spain,Italy,Spain,Santiago Bernabéu,Dennis Viollet,Manchester United,England,9,Centre-Forward


There have been **Seven (7)** occasions in UCL History, where clubs have **won the title** while **playing the finals in their Home country**, with the *last win in 1997*.

In [24]:
df[df["Runner_ups_country"]==df["Host_Country"]].reset_index(drop=True)

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
0,11/12,Bayern Munich,4:5 on pens,Chelsea FC,Chelsea FC,Bayern Munich,1,England,Germany,Germany,Allianz Arena,Lionel Messi,FC Barcelona,Argentina,14,Right Winger
1,10/11,FC Barcelona,3:1,Manchester United,FC Barcelona,Manchester United,2,Spain,England,England,Wembley Stadium,Lionel Messi,FC Barcelona,Argentina,12,Right Winger
2,85/86,Steaua Bucharest,2:0 on pens,FC Barcelona,Steaua Bucharest,FC Barcelona,2,Romania,Spain,Spain,Ramón Sánchez Pizjuán,Torbjörn Nilsson,IFK Göteborg,Sweden,6,Centre-Forward
3,83/84,Liverpool FC,5:3 on pens,AS Roma,Liverpool FC,AS Roma,2,England,Italy,Italy,Stadio Olimpico,Viktor Sokol,Dinamo Minsk,Belarus,6,Centre-Forward
4,55/56,Real Madrid,4:3,Stade Reims,Real Madrid,Stade Reims,1,Spain,France,France,Parc des Princes,Milos Milutinovic,FK Partizan Belgrade,Serbia,8,Centre-Forward


On the other hand, there have been **5 times** where clubs **lost the finals** hosted **in their home country**.

####  Exploring *finals* that went to **Penalties/Extra time**

In [25]:
(df[df["FT_score"]
.str.contains('pens')]
.reset_index(drop=True))

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
0,15/16,Real Madrid,6:4 on pens,Atlético de Madrid,Real Madrid,Atlético de Madrid,2,Spain,Spain,Italy,San Siro,Cristiano Ronaldo,Real Madrid,Portugal,16,Centre-Forward
1,11/12,Bayern Munich,4:5 on pens,Chelsea FC,Chelsea FC,Bayern Munich,1,England,Germany,Germany,Allianz Arena,Lionel Messi,FC Barcelona,Argentina,14,Right Winger
2,07/08,Manchester United,6:5 on pens,Chelsea FC,Manchester United,Chelsea FC,1,England,England,Russia,Luzhniki Stadium,Cristiano Ronaldo,Manchester United,Portugal,8,Centre-Forward
3,04/05,AC Milan,5:6 on pens,Liverpool FC,Liverpool FC,AC Milan,1,England,Italy,Turkey,Atatürk Olympic Stadium,Ruud van Nistelrooy,Manchester United,Netherlands,8,Centre-Forward
4,02/03,Juventus FC,2:3 on pens,AC Milan,AC Milan,Juventus FC,1,Italy,Italy,England,Old Trafford,Ruud van Nistelrooy,Manchester United,Netherlands,12,Centre-Forward
5,00/01,Bayern Munich,5:4 on pens,Valencia CF,Bayern Munich,Valencia CF,1,Germany,Spain,Italy,San Siro,Andriy Shevchenko,AC Milan,Ukraine,9,Centre-Forward
6,95/96,Juventus FC,5:3 on pens,Ajax Amsterdam,Juventus FC,Ajax Amsterdam,2,Italy,Netherlands,Italy,Stadio Olimpico,Jari Litmanen,Ajax Amsterdam,Finland,9,Attacking Midfield
7,90/91,Red Star Belgrade,5:3 on pens,Olympique Marseille,Red Star Belgrade,Olympique Marseille,2,Yugoslavia,France,Italy,Stadio San Nicola,"Peter Pacult, Jean-Pierre Papin, Darko Pancev","FC Swarovski Tirol, Olympique Marseille, Red S...","Austria, France, North Macedonia",6,"Sturm, Centre-Forward, Centre-Forward"
8,87/88,PSV Eindhoven,6:5 on pens,SL Benfica,PSV Eindhoven,SL Benfica,1,Netherlands,Portugal,Germany,Neckarstadion,"Míchel, Jean-Marc Ferreri","Real Madrid, FC Girondins Bordeaux","Spain, France",4,"Right Midfield, Attacking Midfield"
9,85/86,Steaua Bucharest,2:0 on pens,FC Barcelona,Steaua Bucharest,FC Barcelona,2,Romania,Spain,Spain,Ramón Sánchez Pizjuán,Torbjörn Nilsson,IFK Göteborg,Sweden,6,Centre-Forward


A total of **11 UCL Finals** went to **penalties**

In [26]:
(df
[df["FT_score"].str.contains('pens')]
.reset_index(drop=True)
["Winning_team"].value_counts())
## teams and no. of times they won ucl on penalties

Liverpool FC         2
Real Madrid          1
Chelsea FC           1
Manchester United    1
AC Milan             1
Bayern Munich        1
Juventus FC          1
Red Star Belgrade    1
PSV Eindhoven        1
Steaua Bucharest     1
Name: Winning_team, dtype: int64

In [27]:
(df
[df["FT_score"].str.contains('pens')]
.reset_index(drop=True)
["Losing_team"].value_counts())
#clubs that lost on penalties

Atlético de Madrid     1
Bayern Munich          1
Chelsea FC             1
AC Milan               1
Juventus FC            1
Valencia CF            1
Ajax Amsterdam         1
Olympique Marseille    1
SL Benfica             1
FC Barcelona           1
AS Roma                1
Name: Losing_team, dtype: int64

In [28]:
(df
[df["FT_score"]
.str.contains('AET')]
.reset_index(drop=True))

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
0,13/14,Real Madrid,4:1 AET,Atlético de Madrid,Real Madrid,Atlético de Madrid,3,Spain,Spain,Portugal,Estádio da Luz,Cristiano Ronaldo,Real Madrid,Portugal,17,Centre-Forward
1,91/92,FC Barcelona,1:0 AET,UC Sampdoria,FC Barcelona,UC Sampdoria,1,Spain,Italy,England,Wembley Stadium,Jean-Pierre Papin,Olympique Marseille,France,7,Centre-Forward
2,69/70,RV & AV Feijenoord,2:1 AET,Celtic FC,RV & AV Feijenoord,Celtic FC,1,Netherlands,Scotland,Italy,San Siro,Ove Kindvall,Feyenoord Rotterdam,Sweden,7,Centre-Forward
3,67/68,Manchester United,4:1 AET,SL Benfica,Manchester United,SL Benfica,3,England,Portugal,England,Wembley Stadium,Eusébio,SL Benfica,Portugal,6,Centre-Forward
4,57/58,Real Madrid,3:2 AET,AC Milan,Real Madrid,AC Milan,1,Spain,Italy,Belgium,Heysel Stadium,Alfredo di Stéfano,Real Madrid,Spain,10,Second Striker


In only **5 seasons**, the finals were decided in **Extra time**

In [29]:
(df
[df["FT_score"]
.str.contains('AET')]
.reset_index(drop=True)
["Winning_team"].value_counts())
#teams that won in extra time

Real Madrid           2
FC Barcelona          1
RV & AV Feijenoord    1
Manchester United     1
Name: Winning_team, dtype: int64

#### Home team vs away team performance in ucl finals
NOTE: For administrative purposes, there is always a "home" team at a final -- even if it is held at a neutral venue.
It helps with scenarios such as kit clashes by dictating that the "away" side should wear their alternative strip.

In [30]:
(len(df
     [df["Home_team"]==df["Winning_team"]]
     .reset_index(drop=True)))

54

Club as **Home team** has a massive **78.2% win** rate in the finals, **winning in 54/69** seasons

In [31]:
(len(df)
-len(df[df["Home_team"]==df["Winning_team"]]
.reset_index(drop=True)))

15

**Away team** in the final won only **15** times

#### Analyzing performance of Countries(w.r.t. clubs) in UCL finals

In [32]:
df["Winners_country"].value_counts()

Spain          20
England        15
Italy          12
Germany         8
Netherlands     6
Portugal        4
France          1
Yugoslavia      1
Romania         1
Scotland        1
Name: Winners_country, dtype: int64

In [33]:
df["Runner_ups_country"].value_counts()

Italy          17
Germany        11
England        11
Spain          11
France          6
Portugal        5
Netherlands     2
Romania         1
Sweden          1
Belgium         1
Greece          1
Scotland        1
Yugoslavia      1
Name: Runner_ups_country, dtype: int64

In [34]:
(df["Winners_country"].value_counts()
 .add(df["Runner_ups_country"].value_counts(), fill_value=0)
 .astype(int)
 .sort_values(ascending=False))

Spain          31
Italy          29
England        26
Germany        19
Portugal        9
Netherlands     8
France          7
Romania         2
Scotland        2
Yugoslavia      2
Belgium         1
Greece          1
Sweden          1
dtype: int32

In [35]:
(df["Winners_country"].value_counts()
               .div(df["Winners_country"].value_counts()
               .add(df["Runner_ups_country"].value_counts(), fill_value=0))
               .fillna(0)
               .sort_values(ascending=False)
               .mul(100)
               .round(2)
              )
#calculating win % in finals(wrt clubs from a country)

Netherlands    75.00
Spain          64.52
England        57.69
Romania        50.00
Scotland       50.00
Yugoslavia     50.00
Portugal       44.44
Germany        42.11
Italy          41.38
France         14.29
Belgium         0.00
Greece          0.00
Sweden          0.00
dtype: float64

#### Key Findings:
- **Spanish** clubs have won with **most ucl titles(20)** combined whilst **Italian clubs** have **lost** the UCL Final the most times(**17**).
- Most appearences in UCL finals:
    1. Spain (31)
    2. Italy (29)
    3. England(26)
- Highest Win percentage:
    1. Netherlands     75.00 %
    2. Spain           64.52 %
    3. England         57.69 %

### Analyzing top scorer's attributes

In [33]:
df.sample(3)

,Season,Home_team,FT_score,Away_team,Winning_team,Losing_team,GD,Winners_country,Runner_ups_country,Host_Country,Finals_stadium,Top_scorer,Top_scorer_club,Top_scorer_country,Top_scorer_goals,Top_scorer_position
65,58/59,Real Madrid,2:0,Stade Reims,Real Madrid,Stade Reims,2,Spain,France,Germany,Neckarstadion,Just Fontaine,Stade Reims,France,10,Centre-Forward
56,67/68,Manchester United,4:1 AET,SL Benfica,Manchester United,SL Benfica,3,England,Portugal,England,Wembley Stadium,Eusébio,SL Benfica,Portugal,6,Centre-Forward
46,77/78,Liverpool FC,1:0,Club Brugge KV,Liverpool FC,Club Brugge KV,1,England,Belgium,England,Wembley Stadium,Allan Simonsen,Borussia Mönchengladbach,Denmark,5,Second Striker


In [37]:
(df["Top_scorer"]
.str.split(', ')
.explode() #  Expands lists of players into separate rows.
.value_counts()
)
#players and no. of times they finsihed as highest goalscorer of a season

Cristiano Ronaldo        7
Lionel Messi             6
Gerd Müller              4
Ruud van Nistelrooy      3
Andriy Shevchenko        3
Jean-Pierre Papin        3
Eusébio                  3
Ferenc Puskás            3
Romário                  2
Alfredo di Stéfano       2
Torbjörn Nilsson         2
Erling Haaland           2
Antonis Antoniadis       1
Allan Simonsen           1
Claudio Sulser           1
Jupp Heynckes            1
Eduard Markarov          1
Silvester Takac          1
Sören Lerby              1
Harry Kane               1
Paul Van Himst           1
Ove Kindvall             1
Denis Law                1
Graeme Souness           1
Flórián Albert           1
Sandro Mazzola           1
Vladimir Kovacevic       1
José Altafini            1
José Águas               1
Just Fontaine            1
Dennis Viollet           1
Karl-Heinz Rummenigge    1
Borislav Cvetkovic       1
Dieter Hoeneß            1
Michel Platini           1
Karim Benzema            1
Robert Lewandowski       1
N

In [4]:
(df.assign
 (Top_scorer_club=df['Top_scorer_club']
  .apply(lambda x: x.split(', ') if isinstance(x, str) else x)) 
  .explode('Top_scorer_club') 
  .query("Winning_team == Top_scorer_club")
  .reset_index(drop=True)
  )[["Season","Winning_team","Top_scorer","Top_scorer_goals"]]
#seasons where top scorer has won the UCL

,Season,Winning_team,Top_scorer,Top_scorer_goals
0,22/23,Manchester City,Erling Haaland,12
1,21/22,Real Madrid,Karim Benzema,15
2,19/20,Bayern Munich,Robert Lewandowski,15
3,17/18,Real Madrid,Cristiano Ronaldo,15
4,16/17,Real Madrid,Cristiano Ronaldo,12
5,15/16,Real Madrid,Cristiano Ronaldo,16
6,14/15,FC Barcelona,"Cristiano Ronaldo, Lionel Messi, Neymar",10
7,14/15,FC Barcelona,"Cristiano Ronaldo, Lionel Messi, Neymar",10
8,13/14,Real Madrid,Cristiano Ronaldo,17
9,10/11,FC Barcelona,Lionel Messi,12


In **25/69** seasons (roughly one-third), the **top scorer** has gone on to **win** the **UCL**.

In [13]:
(df.assign
 (Top_scorer_club=df['Top_scorer_club']
  .apply(lambda x: x.split(', ') if isinstance(x, str) else x)) 
  .explode('Top_scorer_club') 
  .query("Losing_team == Top_scorer_club")
  .reset_index(drop=True)
  )[["Season","Losing_team","Top_scorer","Top_scorer_goals"]]
#seasons where top scorer has lost the UCL final

,Season,Losing_team,Top_scorer,Top_scorer_goals
0,03/04,AS Monaco,Fernando Morientes,9
1,97/98,Juventus FC,Alessandro Del Piero,10
2,95/96,Ajax Amsterdam,Jari Litmanen,9
3,93/94,FC Barcelona,Ronald Koeman,8
4,90/91,Olympique Marseille,"Peter Pacult, Jean-Pierre Papin, Darko Pancev",6
5,82/83,Juventus FC,Michel Platini,5
6,81/82,Bayern Munich,Dieter Hoeneß,7
7,70/71,Panathinaikos FC,Antonis Antoniadis,10
8,67/68,SL Benfica,Eusébio,6
9,64/65,SL Benfica,Eusébio,9


Contrarily, there have been **14 seasons** where the **top scorer** finished as the **runner up**.

In [18]:
(df.assign
 (Top_scorer_club=df['Top_scorer_club']
  .apply(lambda x: x.split(', ') if isinstance(x, str) else x)) 
  .explode('Top_scorer_club') 
  .query("Losing_team != Top_scorer_club & Winning_team != Top_scorer_club")
  .reset_index(drop=True)
  )[["Season","Winning_team","Losing_team","Top_scorer_club","Top_scorer","Top_scorer_goals"]]

,Season,Winning_team,Losing_team,Top_scorer_club,Top_scorer,Top_scorer_goals
0,23/24,Real Madrid,Borussia Dortmund,Bayern Munich,"Harry Kane, Kylian Mbappé",8
1,23/24,Real Madrid,Borussia Dortmund,Paris Saint-Germain,"Harry Kane, Kylian Mbappé",8
2,20/21,Chelsea FC,Manchester City,Borussia Dortmund,Erling Haaland,10
3,18/19,Liverpool FC,Tottenham Hotspur,FC Barcelona,Lionel Messi,12
4,14/15,FC Barcelona,Juventus FC,Real Madrid,"Cristiano Ronaldo, Lionel Messi, Neymar",10
5,12/13,Bayern Munich,Borussia Dortmund,Real Madrid,Cristiano Ronaldo,12
6,11/12,Chelsea FC,Bayern Munich,FC Barcelona,Lionel Messi,14
7,09/10,Inter Milan,Bayern Munich,FC Barcelona,Lionel Messi,8
8,05/06,FC Barcelona,Arsenal FC,AC Milan,Andriy Shevchenko,9
9,04/05,Liverpool FC,AC Milan,Manchester United,Ruud van Nistelrooy,8


There have been **46 top scorers** who **did not reach** the *UCL final*

In [40]:
(df.assign
 (Top_scorer_club=df['Top_scorer_country']
  .apply(lambda x: x.split(', ') if isinstance(x, str) else x)) 
  .explode('Top_scorer_country') 
  .query("Host_Country == Top_scorer_country")
  .reset_index(drop=True)
  )[["Season","Host_Country","Top_scorer","Top_scorer_goals"]]


,Season,Host_Country,Top_scorer,Top_scorer_goals
0,21/22,France,Karim Benzema,15
1,13/14,Portugal,Cristiano Ronaldo,17


Only **2/69 seasons** have seen the **Top scorer** *lift the UCL Trophy* in his **home country**

In [41]:
(df["Top_scorer_position"]
.str.split(', ')
.explode() 
.value_counts()
)
#analyzing top scorer's position 

Centre-Forward        59
Second Striker         7
Right Winger           6
Attacking Midfield     6
Left Winger            2
Central Midfield       2
Sweeper                1
Sturm                  1
Right Midfield         1
Name: Top_scorer_position, dtype: int64

In [43]:
df["Top_scorer_goals"].describe()

count    69.000000
mean      9.043478
std       2.947755
min       4.000000
25%       7.000000
50%       9.000000
75%      10.000000
max      17.000000
Name: Top_scorer_goals, dtype: float64

In [44]:
import psutil
process = psutil.Process(os.getpid())

# Check memory usage in MB
memory_usage = process.memory_info().rss / (1024 * 1024)  # in MB
print(f"Current memory usage: {memory_usage:.2f} MB")

Current memory usage: 130.80 MB
